In [1]:
#!/usr/bin/env python3
"""
apply_zp_to_reference_stars.py

Aplica los zero points a las estrellas de referencia y compara con:
1. Magnitudes sintéticas de Gaia XP (mag_F378, etc.)
2. Magnitudes broad-band de Gaia (G, BP, RP)

Esto valida la calidad de nuestra calibración fotométrica.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import glob
import os

# Configuración de estilo
plt.style.use('default')
sns.set_palette("colorblind")

def load_zero_points(zp_file):
    """Carga los zero points desde el archivo CSV"""
    zp_df = pd.read_csv(zp_file)
    print("Zero points cargados:")
    print(zp_df)
    return zp_df

def apply_zero_points_to_stars(star_file, zp_df):
    """
    Aplica los zero points a las estrellas de referencia y calcula magnitudes calibradas
    """
    # Cargar datos de estrellas
    stars_df = pd.read_csv(star_file)
    field_name = os.path.basename(star_file).split('_')[0]
    
    print(f"\nProcesando campo: {field_name}")
    print(f"Estrellas en catálogo: {len(stars_df)}")
    
    # Obtener zero points para este campo
    field_zp = zp_df[zp_df['field'] == field_name]
    if len(field_zp) == 0:
        print(f"⚠️  No se encontraron zero points para {field_name}")
        return None
    
    field_zp = field_zp.iloc[0]
    
    # Filtros SPLUS
    splus_filters = ['F378', 'F395', 'F410', 'F430', 'F515', 'F660', 'F861']
    
    # Aplicar zero points a cada filtro
    for filter_name in splus_filters:
        inst_mag_col = f'mag_inst_3.0_{filter_name}'
        calib_mag_col = f'mag_calib_{filter_name}'
        synth_mag_col = f'mag_{filter_name}'  # Magnitud sintética de Gaia XP
        
        # Verificar que tenemos la columna de magnitud instrumental
        if inst_mag_col not in stars_df.columns:
            print(f"⚠️  Columna {inst_mag_col} no encontrada")
            continue
        
        # Aplicar zero point: mag_calib = mag_inst + ZP
        zp_value = field_zp[filter_name]
        stars_df[calib_mag_col] = stars_df[inst_mag_col] + zp_value
        
        print(f"✅ {filter_name}: ZP = {zp_value:.3f}, "
              f"mag_inst range = [{stars_df[inst_mag_col].min():.2f}, {stars_df[inst_mag_col].max():.2f}], "
              f"mag_calib range = [{stars_df[calib_mag_col].min():.2f}, {stars_df[calib_mag_col].max():.2f}]")
    
    return stars_df

def analyze_calibration_quality(calibrated_stars, field_name):
    """
    Analiza la calidad de la calibración comparando con magnitudes de referencia
    """
    splus_filters = ['F378', 'F395', 'F410', 'F430', 'F515', 'F660', 'F861']
    
    # Crear figura para análisis
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    axes = axes.ravel()
    
    results = []
    
    for i, filter_name in enumerate(splus_filters):
        if i >= len(axes):
            break
            
        calib_mag_col = f'mag_calib_{filter_name}'
        synth_mag_col = f'mag_{filter_name}'  # Magnitud sintética de Gaia XP
        
        # Verificar que tenemos ambas columnas
        if calib_mag_col not in calibrated_stars.columns or synth_mag_col not in calibrated_stars.columns:
            continue
        
        # Filtrar datos válidos
        valid_mask = (
            calibrated_stars[calib_mag_col].notna() & 
            calibrated_stars[synth_mag_col].notna() &
            np.isfinite(calibrated_stars[calib_mag_col]) & 
            np.isfinite(calibrated_stars[synth_mag_col]) &
            (calibrated_stars[calib_mag_col] < 30) &  # Magnitudes razonables
            (calibrated_stars[synth_mag_col] < 30) &
            (calibrated_stars[calib_mag_col] > 10) &
            (calibrated_stars[synth_mag_col] > 10)
        )
        
        calib_mags = calibrated_stars.loc[valid_mask, calib_mag_col]
        synth_mags = calibrated_stars.loc[valid_mask, synth_mag_col]
        
        if len(calib_mags) < 5:
            continue
        
        # Calcular diferencias
        differences = calib_mags - synth_mags
        
        # Estadísticas
        mean_diff = np.mean(differences)
        median_diff = np.median(differences)
        std_diff = np.std(differences)
        mad_diff = np.median(np.abs(differences - median_diff))
        correlation = stats.pearsonr(synth_mags, calib_mags)[0]
        
        # Ajuste lineal
        try:
            slope, intercept, r_value, p_value, std_err = stats.linregress(synth_mags, calib_mags)
            r_squared = r_value**2
        except:
            slope, intercept, r_squared, std_err = 1.0, 0.0, 0.0, 0.0
        
        results.append({
            'filter': filter_name,
            'n_stars': len(calib_mags),
            'correlation': correlation,
            'r_squared': r_squared,
            'slope': slope,
            'intercept': intercept,
            'mean_diff': mean_diff,
            'median_diff': median_diff,
            'std_diff': std_diff,
            'mad_diff': mad_diff
        })
        
        # Scatter plot
        ax = axes[i]
        sc = ax.scatter(synth_mags, calib_mags, alpha=0.6, s=20, c=differences,
                       cmap='coolwarm', vmin=-1, vmax=1)
        
        # Línea 1:1
        x_range = np.linspace(min(synth_mags), max(synth_mags), 100)
        ax.plot(x_range, x_range, 'k--', alpha=0.7, label='1:1')
        
        # Línea ajustada
        ax.plot(x_range, slope*x_range + intercept, 'r-', 
                label=f'y = {slope:.3f}x + {intercept:.3f}')
        
        ax.set_xlabel(f'Gaia XP Synthetic {filter_name}')
        ax.set_ylabel(f'SPLUS Calibrated {filter_name}')
        ax.set_title(f'{filter_name}\nΔ_med = {median_diff:.3f}, r = {correlation:.3f}')
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
        
        # Barra de color para diferencias
        plt.colorbar(sc, ax=ax, label='SPLUS - Gaia XP')
    
    # Ocultar ejes vacíos
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle(f'Validación de Calibración - {field_name}\n(SPLUS Calibrado vs Gaia XP Sintético)', 
                 fontsize=16, y=0.98)
    plt.tight_layout()
    plt.savefig(f'calibration_validation_{field_name}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return pd.DataFrame(results)

def compare_with_gaia_photometry(calibrated_stars, field_name):
    """
    Compara las magnitudes calibradas con las magnitudes broad-band de Gaia
    """
    # Correspondencias aproximadas entre filtros SPLUS y Gaia
    gaia_comparisons = [
        ('F515', 'gaia_phot_g_mean_mag', 'F515 vs Gaia G'),
        ('F660', 'gaia_phot_rp_mean_mag', 'F660 vs Gaia RP (aproximado)'),
        ('F861', 'gaia_phot_rp_mean_mag', 'F861 vs Gaia RP (aproximado)'),
    ]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    gaia_results = []
    
    for i, (splus_filter, gaia_mag_col, title) in enumerate(gaia_comparisons):
        if i >= len(axes):
            break
            
        calib_mag_col = f'mag_calib_{splus_filter}'
        
        if calib_mag_col not in calibrated_stars.columns or gaia_mag_col not in calibrated_stars.columns:
            continue
        
        # Filtrar datos válidos
        valid_mask = (
            calibrated_stars[calib_mag_col].notna() & 
            calibrated_stars[gaia_mag_col].notna() &
            np.isfinite(calibrated_stars[calib_mag_col]) & 
            np.isfinite(calibrated_stars[gaia_mag_col]) &
            (calibrated_stars[calib_mag_col] < 30) &
            (calibrated_stars[gaia_mag_col] < 30) &
            (calibrated_stars[calib_mag_col] > 10) &
            (calibrated_stars[gaia_mag_col] > 10)
        )
        
        splus_mags = calibrated_stars.loc[valid_mask, calib_mag_col]
        gaia_mags = calibrated_stars.loc[valid_mask, gaia_mag_col]
        
        if len(splus_mags) < 5:
            continue
        
        # Calcular diferencias
        differences = splus_mags - gaia_mags
        
        # Estadísticas
        mean_diff = np.mean(differences)
        median_diff = np.median(differences)
        std_diff = np.std(differences)
        correlation = stats.pearsonr(gaia_mags, splus_mags)[0]
        
        gaia_results.append({
            'splus_filter': splus_filter,
            'gaia_filter': gaia_mag_col,
            'n_stars': len(splus_mags),
            'correlation': correlation,
            'mean_diff': mean_diff,
            'median_diff': median_diff,
            'std_diff': std_diff
        })
        
        # Scatter plot
        ax = axes[i]
        sc = ax.scatter(gaia_mags, splus_mags, alpha=0.6, s=20, c=differences,
                       cmap='coolwarm', vmin=-2, vmax=2)
        
        # Línea 1:1
        x_range = np.linspace(min(gaia_mags), max(gaia_mags), 100)
        ax.plot(x_range, x_range, 'k--', alpha=0.7, label='1:1')
        
        ax.set_xlabel(f'Gaia {gaia_mag_col}')
        ax.set_ylabel(f'SPLUS {splus_filter} (calibrado)')
        ax.set_title(f'{title}\nΔ_med = {median_diff:.3f}, r = {correlation:.3f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.colorbar(sc, ax=ax, label='SPLUS - Gaia')
    
    plt.suptitle(f'Comparación con Gaia Broad-band - {field_name}', fontsize=14)
    plt.tight_layout()
    plt.savefig(f'gaia_comparison_{field_name}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return pd.DataFrame(gaia_results)

def main():
    """Función principal"""
    # Archivos de entrada
    zp_file = '../anac_data/Results/all_fields_zero_points_splus_format_no_aper_corr.csv'
    star_files = [
        '../anac_data/CenA01_gaia_xp_matches_no_aper_corr.csv',
        '../anac_data/CenA02_gaia_xp_matches_no_aper_corr.csv'
    ]
    
    # Cargar zero points
    zp_df = load_zero_points(zp_file)
    
    all_validation_results = []
    all_gaia_results = []
    
    for star_file in star_files:
        if not os.path.exists(star_file):
            print(f"⚠️  Archivo no encontrado: {star_file}")
            continue
        
        # Aplicar zero points
        calibrated_stars = apply_zero_points_to_stars(star_file, zp_df)
        if calibrated_stars is None:
            continue
        
        field_name = os.path.basename(star_file).split('_')[0]
        
        # Guardar estrellas calibradas
        output_file = f'{field_name}_calibrated_stars.csv'
        calibrated_stars.to_csv(output_file, index=False)
        print(f"✅ Estrellas calibradas guardadas en: {output_file}")
        
        # Analizar calidad vs Gaia XP sintético
        validation_results = analyze_calibration_quality(calibrated_stars, field_name)
        if not validation_results.empty:
            validation_results['field'] = field_name
            all_validation_results.append(validation_results)
            
            print(f"\n📊 RESULTADOS VALIDACIÓN - {field_name}:")
            print("="*50)
            print(validation_results.to_string(index=False))
        
        # Comparar con Gaia broad-band
        gaia_results = compare_with_gaia_photometry(calibrated_stars, field_name)
        if not gaia_results.empty:
            gaia_results['field'] = field_name
            all_gaia_results.append(gaia_results)
            
            print(f"\n📊 COMPARACIÓN GAIA - {field_name}:")
            print("="*50)
            print(gaia_results.to_string(index=False))
    
    # Guardar resultados consolidados
    if all_validation_results:
        consolidated_validation = pd.concat(all_validation_results, ignore_index=True)
        consolidated_validation.to_csv('calibration_validation_summary.csv', index=False)
        print("\n✅ Resumen de validación guardado en: calibration_validation_summary.csv")
    
    if all_gaia_results:
        consolidated_gaia = pd.concat(all_gaia_results, ignore_index=True)
        consolidated_gaia.to_csv('gaia_comparison_summary.csv', index=False)
        print("✅ Resumen de comparación con Gaia guardado en: gaia_comparison_summary.csv")
    
    print("\n" + "="*60)
    print("ANÁLISIS COMPLETADO")
    print("="*60)
    print("Archivos generados:")
    print("- [field]_calibrated_stars.csv : Estrellas con magnitudes calibradas")
    print("- calibration_validation_[field].png : Gráficos de validación vs Gaia XP")
    print("- gaia_comparison_[field].png : Gráficos de comparación con Gaia broad-band")
    print("- calibration_validation_summary.csv : Resumen estadístico")
    print("- gaia_comparison_summary.csv : Resumen comparación Gaia")

if __name__ == '__main__':
    main()

Zero points cargados:
    field       F378       F395       F410       F430       F515       F660  \
0  CenA02  19.679261  19.783760  20.732207  20.913429  21.086574  20.626191   
1  CenA01  19.648163  19.683442  20.699321  20.896421  21.097279  20.628396   

        F861  
0  21.156307  
1  21.166124  

Procesando campo: CenA01
Estrellas en catálogo: 396
✅ F378: ZP = 19.648, mag_inst range = [-4.38, -1.75], mag_calib range = [15.27, 17.90]
✅ F395: ZP = 19.683, mag_inst range = [-4.53, -1.94], mag_calib range = [15.15, 17.74]
✅ F410: ZP = 20.699, mag_inst range = [-5.99, -3.64], mag_calib range = [14.71, 17.06]
✅ F430: ZP = 20.896, mag_inst range = [-6.26, -3.90], mag_calib range = [14.63, 16.99]
✅ F515: ZP = 21.097, mag_inst range = [-6.86, -4.74], mag_calib range = [14.24, 16.36]
✅ F660: ZP = 20.628, mag_inst range = [-6.78, -4.71], mag_calib range = [13.84, 15.92]
✅ F861: ZP = 21.166, mag_inst range = [-7.66, -5.43], mag_calib range = [13.50, 15.74]
✅ Estrellas calibradas guardadas 

In [2]:
#!/usr/bin/env python3
"""
analyze_offsets_vs_stellar_properties.py

Analiza los offsets entre SPLUS y Gaia en función de:
- Color (BP-RP) de Gaia
- Tipo espectral (proxy por color)
- Magnitud (brillo)
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def analyze_offsets_by_color(calibrated_stars, field_name):
    """
    Analiza los offsets en función del color (BP-RP) de Gaia
    """
    # Calcular color BP-RP
    calibrated_stars['gaia_bp_rp'] = calibrated_stars['gaia_phot_bp_mean_mag'] - calibrated_stars['gaia_phot_rp_mean_mag']
    
    # Filtros para análisis
    filter_analysis = [
        ('F515', 'gaia_phot_g_mean_mag', 'F515 vs Gaia G'),
        ('F660', 'gaia_phot_rp_mean_mag', 'F660 vs Gaia RP'),
        ('F861', 'gaia_phot_rp_mean_mag', 'F861 vs Gaia RP')
    ]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    results = []
    
    for i, (splus_filter, gaia_filter, title) in enumerate(filter_analysis):
        calib_mag_col = f'mag_calib_{splus_filter}'
        
        if calib_mag_col not in calibrated_stars.columns or gaia_filter not in calibrated_stars.columns:
            continue
        
        # Filtrar datos válidos
        valid_mask = (
            calibrated_stars[calib_mag_col].notna() & 
            calibrated_stars[gaia_filter].notna() &
            calibrated_stars['gaia_bp_rp'].notna() &
            np.isfinite(calibrated_stars[calib_mag_col]) & 
            np.isfinite(calibrated_stars[gaia_filter]) &
            np.isfinite(calibrated_stars['gaia_bp_rp']) &
            (calibrated_stars[calib_mag_col] < 30) &
            (calibrated_stars[gaia_filter] < 30) &
            (calibrated_stars['gaia_bp_rp'] > -1) &  # Colores razonables
            (calibrated_stars['gaia_bp_rp'] < 4)
        )
        
        if valid_mask.sum() < 10:
            continue
        
        splus_mags = calibrated_stars.loc[valid_mask, calib_mag_col]
        gaia_mags = calibrated_stars.loc[valid_mask, gaia_filter]
        colors = calibrated_stars.loc[valid_mask, 'gaia_bp_rp']
        differences = splus_mags - gaia_mags
        
        # Scatter: Offset vs Color
        ax1 = axes[0, i]
        sc1 = ax1.scatter(colors, differences, alpha=0.6, s=20, c=gaia_mags, cmap='viridis')
        ax1.axhline(0, color='red', linestyle='--', alpha=0.7, label='Offset cero')
        
        # Ajuste polinómico para ver tendencia
        try:
            z = np.polyfit(colors, differences, 2)  # Ajuste cuadrático
            x_range = np.linspace(min(colors), max(colors), 100)
            ax1.plot(x_range, np.poly1d(z)(x_range), 'r-', linewidth=2, 
                    label=f'Tendencia polinómica')
        except:
            pass
        
        ax1.set_xlabel('Color Gaia (BP-RP)')
        ax1.set_ylabel('Δ (SPLUS - Gaia)')
        ax1.set_title(f'{title}\nOffset vs Color')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        plt.colorbar(sc1, ax=ax1, label='Magnitud Gaia')
        
        # Histograma de offsets
        ax2 = axes[1, i]
        ax2.hist(differences, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        ax2.axvline(np.median(differences), color='red', linestyle='--', 
                   label=f'Mediana: {np.median(differences):.3f}')
        ax2.set_xlabel('Δ (SPLUS - Gaia)')
        ax2.set_ylabel('Frecuencia')
        ax2.set_title(f'Distribución de Offsets - {splus_filter}')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Estadísticas por rango de color
        color_bins = np.quantile(colors, [0, 0.33, 0.67, 1.0])
        for j in range(len(color_bins)-1):
            bin_mask = (colors >= color_bins[j]) & (colors < color_bins[j+1])
            if bin_mask.sum() > 5:
                bin_median = np.median(differences[bin_mask])
                bin_std = np.std(differences[bin_mask])
                results.append({
                    'field': field_name,
                    'filter': splus_filter,
                    'color_bin': f'{color_bins[j]:.2f}-{color_bins[j+1]:.2f}',
                    'median_offset': bin_median,
                    'std_offset': bin_std,
                    'n_stars': bin_mask.sum()
                })
    
    plt.suptitle(f'Análisis de Offsets vs Color - {field_name}', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'offsets_color_analysis_{field_name}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return pd.DataFrame(results)

def analyze_offsets_by_magnitude(calibrated_stars, field_name):
    """
    Analiza los offsets en función de la magnitud (brillo)
    """
    filter_analysis = [
        ('F515', 'gaia_phot_g_mean_mag'),
        ('F660', 'gaia_phot_rp_mean_mag'), 
        ('F861', 'gaia_phot_rp_mean_mag')
    ]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, (splus_filter, gaia_filter) in enumerate(filter_analysis):
        if i >= len(axes):
            break
            
        calib_mag_col = f'mag_calib_{splus_filter}'
        
        if calib_mag_col not in calibrated_stars.columns or gaia_filter not in calibrated_stars.columns:
            continue
        
        valid_mask = (
            calibrated_stars[calib_mag_col].notna() & 
            calibrated_stars[gaia_filter].notna() &
            np.isfinite(calibrated_stars[calib_mag_col]) & 
            np.isfinite(calibrated_stars[gaia_filter]) &
            (calibrated_stars[calib_mag_col] < 30) &
            (calibrated_stars[gaia_filter] < 30)
        )
        
        if valid_mask.sum() < 10:
            continue
        
        splus_mags = calibrated_stars.loc[valid_mask, calib_mag_col]
        gaia_mags = calibrated_stars.loc[valid_mask, gaia_filter]
        differences = splus_mags - gaia_mags
        
        ax = axes[i]
        sc = ax.scatter(gaia_mags, differences, alpha=0.6, s=20, 
                       c=calibrated_stars.loc[valid_mask, 'gaia_bp_rp'], cmap='coolwarm')
        ax.axhline(0, color='red', linestyle='--', alpha=0.7)
        
        # Ajuste lineal para ver tendencia con magnitud
        try:
            z = np.polyfit(gaia_mags, differences, 1)
            x_range = np.linspace(min(gaia_mags), max(gaia_mags), 100)
            ax.plot(x_range, np.poly1d(z)(x_range), 'k-', linewidth=2,
                   label=f'Tendencia: {z[0]:.3f}x + {z[1]:.3f}')
        except:
            pass
        
        ax.set_xlabel(f'Magnitud Gaia')
        ax.set_ylabel('Δ (SPLUS - Gaia)')
        ax.set_title(f'{splus_filter} - Dependencia con Magnitud')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.colorbar(sc, ax=ax, label='Color Gaia (BP-RP)')
    
    plt.suptitle(f'Offsets vs Magnitud - {field_name}', fontsize=14)
    plt.tight_layout()
    plt.savefig(f'offsets_magnitude_analysis_{field_name}.png', dpi=300, bbox_inches='tight')
    plt.close()

def create_transformation_equations(calibrated_stars_list, output_file='color_transformations.csv'):
    """
    Deriva ecuaciones de transformación entre SPLUS y Gaia
    """
    all_results = []
    
    for calibrated_stars, field_name in calibrated_stars_list:
        calibrated_stars['gaia_bp_rp'] = calibrated_stars['gaia_phot_bp_mean_mag'] - calibrated_stars['gaia_phot_rp_mean_mag']
        
        transformations = [
            ('F515', 'gaia_phot_g_mean_mag', 'G'),
            ('F660', 'gaia_phot_rp_mean_mag', 'RP'),
            ('F861', 'gaia_phot_rp_mean_mag', 'RP')
        ]
        
        for splus_filter, gaia_filter, gaia_band in transformations:
            calib_mag_col = f'mag_calib_{splus_filter}'
            
            if calib_mag_col not in calibrated_stars.columns or gaia_filter not in calibrated_stars.columns:
                continue
            
            valid_mask = (
                calibrated_stars[calib_mag_col].notna() & 
                calibrated_stars[gaia_filter].notna() &
                calibrated_stars['gaia_bp_rp'].notna() &
                np.isfinite(calibrated_stars[calib_mag_col]) & 
                np.isfinite(calibrated_stars[gaia_filter]) &
                np.isfinite(calibrated_stars['gaia_bp_rp'])
            )
            
            if valid_mask.sum() < 20:
                continue
            
            splus_mags = calibrated_stars.loc[valid_mask, calib_mag_col]
            gaia_mags = calibrated_stars.loc[valid_mask, gaia_filter]
            colors = calibrated_stars.loc[valid_mask, 'gaia_bp_rp']
            
            # Ajustar: SPLUS = Gaia + a + b*(BP-RP)
            # Es decir: SPLUS - Gaia = a + b*(BP-RP)
            differences = splus_mags - gaia_mags
            
            try:
                # Ajuste lineal: Δ = a + b*(BP-RP)
                coefficients = np.polyfit(colors, differences, 1)
                a, b = coefficients[1], coefficients[0]  # intercept, slope
                
                # Calcular residuals
                predicted = a + b * colors
                residuals = differences - predicted
                rms = np.sqrt(np.mean(residuals**2))
                
                all_results.append({
                    'field': field_name,
                    'splus_filter': splus_filter,
                    'gaia_band': gaia_band,
                    'transformation_equation': f'SPLUS_{splus_filter} = Gaia_{gaia_band} + {a:.4f} + {b:.4f}*(BP-RP)',
                    'constant_term': a,
                    'color_term': b,
                    'rms': rms,
                    'n_stars': len(splus_mags)
                })
                
                print(f"{field_name} - {splus_filter}: SPLUS = Gaia + {a:.4f} + {b:.4f}*(BP-RP) [RMS: {rms:.4f}]")
                
            except Exception as e:
                print(f"Error en ajuste para {field_name} {splus_filter}: {e}")
    
    if all_results:
        transformation_df = pd.DataFrame(all_results)
        transformation_df.to_csv(output_file, index=False)
        print(f"\n✅ Ecuaciones de transformación guardadas en: {output_file}")
        return transformation_df
    else:
        print("❌ No se pudieron derivar ecuaciones de transformación")
        return None

def main():
    """Función principal"""
    # Cargar datos calibrados
    fields_data = []
    
    for field in ['CenA01', 'CenA02']:
        calibrated_file = f'{field}_calibrated_stars.csv'
        try:
            df = pd.read_csv(calibrated_file)
            fields_data.append((df, field))
            print(f"✅ Cargado: {calibrated_file} ({len(df)} estrellas)")
        except FileNotFoundError:
            print(f"❌ No se encontró: {calibrated_file}")
    
    if not fields_data:
        print("❌ No se encontraron archivos calibrados")
        return
    
    # Análisis por campo
    all_color_results = []
    
    for calibrated_stars, field_name in fields_data:
        print(f"\n🔍 Analizando offsets para {field_name}...")
        
        # Análisis por color
        color_results = analyze_offsets_by_color(calibrated_stars, field_name)
        if not color_results.empty:
            all_color_results.append(color_results)
        
        # Análisis por magnitud
        analyze_offsets_by_magnitude(calibrated_stars, field_name)
    
    # Guardar resultados de color
    if all_color_results:
        color_summary = pd.concat(all_color_results, ignore_index=True)
        color_summary.to_csv('offsets_color_summary.csv', index=False)
        print("\n✅ Resumen de offsets por color guardado en: offsets_color_summary.csv")
        
        # Imprimir resumen
        print("\n📊 RESUMEN OFFSETS POR COLOR:")
        print("="*60)
        for field in color_summary['field'].unique():
            print(f"\n{field}:")
            field_data = color_summary[color_summary['field'] == field]
            for filter_name in field_data['filter'].unique():
                filter_data = field_data[field_data['filter'] == filter_name]
                print(f"  {filter_name}:")
                for _, row in filter_data.iterrows():
                    print(f"    Color {row['color_bin']}: Δ_med = {row['median_offset']:.3f} ± {row['std_offset']:.3f} (N={row['n_stars']})")
    
    # Derivar ecuaciones de transformación
    print("\n🧮 DERIVANDO ECUACIONES DE TRANSFORMACIÓN...")
    transformation_eqs = create_transformation_equations(fields_data)
    
    print("\n" + "="*80)
    print("ANÁLISIS COMPLETADO")
    print("="*80)
    print("Los offsets sistemáticos son NORMALES y ESPERADOS debido a:")
    print("1. Diferentes sistemas fotométricos (narrow-band vs broad-band)")
    print("2. Diferentes curvas de respuesta espectral")
    print("3. Diferentes distribuciones de energía espectral de las estrellas")
    print("\n✅ Use las ecuaciones de transformación para comparaciones precisas")

if __name__ == '__main__':
    main()

✅ Cargado: CenA01_calibrated_stars.csv (396 estrellas)
✅ Cargado: CenA02_calibrated_stars.csv (401 estrellas)

🔍 Analizando offsets para CenA01...

🔍 Analizando offsets para CenA02...

✅ Resumen de offsets por color guardado en: offsets_color_summary.csv

📊 RESUMEN OFFSETS POR COLOR:

CenA01:
  F515:
    Color 0.57-0.90: Δ_med = 0.280 ± 0.039 (N=131)
    Color 0.90-1.01: Δ_med = 0.342 ± 0.033 (N=134)
    Color 1.01-1.52: Δ_med = 0.482 ± 0.094 (N=130)
  F660:
    Color 0.57-0.90: Δ_med = 0.454 ± 0.023 (N=131)
    Color 0.90-1.01: Δ_med = 0.468 ± 0.014 (N=134)
    Color 1.01-1.52: Δ_med = 0.532 ± 0.040 (N=130)
  F861:
    Color 0.57-0.90: Δ_med = 0.321 ± 0.023 (N=131)
    Color 0.90-1.01: Δ_med = 0.305 ± 0.013 (N=134)
    Color 1.01-1.52: Δ_med = 0.262 ± 0.029 (N=130)

CenA02:
  F515:
    Color 0.14-0.88: Δ_med = 0.263 ± 0.048 (N=132)
    Color 0.88-0.99: Δ_med = 0.333 ± 0.039 (N=136)
    Color 0.99-1.64: Δ_med = 0.471 ± 0.100 (N=132)
  F660:
    Color 0.14-0.88: Δ_med = 0.438 ± 0.021 (N